In [ ]:
from google.colab import files
files.upload()  # Manually upload kaggle.json when prompted

Saving kaggle.json to kaggle.json


{'kaggle.json': b'{"username":"navyasingh8905","key":"81a236cc8ecaeb4cc321963f82c1afcb"}'}

In [ ]:
import os
import shutil

os.makedirs("/root/.kaggle", exist_ok=True)
shutil.move("kaggle.json", "/root/.kaggle/kaggle.json")
os.chmod("/root/.kaggle/kaggle.json", 600)


In [ ]:
!pip install kaggle
!kaggle datasets download -d msambare/fer2013

Dataset URL: https://www.kaggle.com/datasets/msambare/fer2013
License(s): DbCL-1.0


In [ ]:
import zipfile

with zipfile.ZipFile("fer2013.zip", "r") as zip_ref:
    zip_ref.extractall("dataset")  # Extract to "dataset" folder

In [ ]:
import os

print(os.listdir("dataset"))  # List extracted files

['train', 'test']


In [ ]:
!pip install keras tensorflow opencv-python numpy

import cv2
import numpy as np
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, Dense, Dropout
from tensorflow.keras.preprocessing.image import ImageDataGenerator
import os
import zipfile

# UNZIP FER-2013 dataset (Make sure it is already downloaded)
dataset_path = "fer2013.zip"  # Change if necessary
with zipfile.ZipFile(dataset_path, 'r') as zip_ref:
    zip_ref.extractall("dataset")

# Load dataset paths
train_dir = "dataset/train"
test_dir = "dataset/test"

# Define Image Data Generators for Augmentation
train_datagen = ImageDataGenerator(rescale=1./255, rotation_range=10, horizontal_flip=True)
test_datagen = ImageDataGenerator(rescale=1./255)

# Load Data
train_generator = train_datagen.flow_from_directory(
    train_dir, target_size=(48, 48), batch_size=64, color_mode="grayscale", class_mode="categorical")

test_generator = test_datagen.flow_from_directory(
    test_dir, target_size=(48, 48), batch_size=64, color_mode="grayscale", class_mode="categorical")

# Define the CNN Model
model = Sequential([
    Conv2D(64, (3, 3), activation='relu', input_shape=(48, 48, 1)),
    MaxPooling2D(2, 2),
    Conv2D(128, (3, 3), activation='relu'),
    MaxPooling2D(2, 2),
    Conv2D(256, (3, 3), activation='relu'),
    MaxPooling2D(2, 2),
    Flatten(),
    Dense(256, activation='relu'),
    Dropout(0.5),
    Dense(128, activation='relu'),
    Dense(7, activation='softmax')  # 7 emotions in FER-2013 dataset
])

# Compile Model
model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])

# Train Model
model.fit(train_generator, epochs=10, validation_data=test_generator)

# Save the trained model
model.save("emotion_detector.keras")

print("Model Training Completed and Saved!")


Found 28709 images belonging to 7 classes.
Found 7178 images belonging to 7 classes.
Epoch 1/10
449/449 ━━━━━━━━━━━━━━━━━━━━ 340s 751ms/step - accuracy: 0.2443 - loss: 1.8220 - val_accuracy: 0.3447 - val_loss: 1.6597
Epoch 2/10
449/449 ━━━━━━━━━━━━━━━━━━━━ 337s 750ms/step - accuracy: 0.3505 - loss: 1.6434 - val_accuracy: 0.4393 - val_loss: 1.4584
Epoch 3/10
449/449 ━━━━━━━━━━━━━━━━━━━━ 325s 724ms/step - accuracy: 0.4351 - loss: 1.4707 - val_accuracy: 0.4808 - val_loss: 1.3705
Epoch 4/10
449/449 ━━━━━━━━━━━━━━━━━━━━ 393s 749ms/step - accuracy: 0.4645 - loss: 1.3924 - val_accuracy: 0.5033 - val_loss: 1.2829
Epoch 5/10
449/449 ━━━━━━━━━━━━━━━━━━━━ 331s 738ms/step - accuracy: 0.4842 - loss: 1.3351 - val_accuracy: 0.5166 - val_loss: 1.2605
Epoch 6/10
449/449 ━━━━━━━━━━━━━━━━━━━━ 333s 741ms/step - accuracy: 0.5051 - loss: 1.2843 - val_accuracy: 0.5371 - val_loss: 1.2014
Epoch 7/10
449/449 ━━━━━━━━━━━━━━━━━━━━ 329s 732ms/step - accuracy: 0.5342 - loss: 1.2354 - val_accuracy: 0.5538 - val_loss

In [ ]:
import joblib
from tensorflow.keras.models import load_model

# Load the trained Keras model
model = load_model("emotion_detector.keras")

# Save model weights using joblib
joblib.dump(model.get_weights(), "model_weights.pkl")

# Save the model architecture separately
with open("model_architecture.json", "w") as json_file:
    json_file.write(model.to_json())

print("Model saved successfully!")


Model saved successfully!
